In [1]:
import pandas as pd
# requires install installing:
# * xlrd for xls extension
# * openpyxl for xlsx extension
import csv
import re
#% Dump sheets as csvs from an excel file quickly with pandas
def extract_sheets_as_csvs(fname, out_folder):
    assert fname.endswith('.xlsx'), "Filename must be xlsx format"
    assert out_folder.endswith('/'), "out_folder must like folder1/folder2/ and end with /"
    
    xls = pd.ExcelFile(fname)

    for sheet in xls.sheet_names:
        obj_sheet = pd.read_excel(xls, sheet, header=0, dtype=str)
        obj_sheet.to_csv(out_folder+sheet+'.csv', index=False)
    return True

In [2]:
assert extract_sheets_as_csvs('data_dict_tables.xlsx', 'tables/')

In [3]:
from glob import glob
the_tables = [table for table in glob('tables/*.csv') if '_RELATIONSHIPS.csv' not in table]
the_tables

['tables/TABLE.csv',
 'tables/TABLE_QA.csv',
 'tables/TABLE_AUTOMATION.csv',
 'tables/GROUP.csv',
 'tables/PROJECTS.csv',
 'tables/APP.csv']

In [4]:
df = pd.read_csv('tables/_RELATIONSHIPS.csv')
df

,first-entity,left-rel,line,right-rel,second-entity,relationship-label
0,PROJECTS,}o,--,o{,APP,NaN
1,APP,}o,--,o{,GROUP,NaN
2,GROUP,}o,--,o{,TABLE,NaN
3,GROUP,}o,--,o{,TABLE_AUTOMATION,NaN
4,TABLE_AUTOMATION,}o,--,o{,TABLE_QA,NaN
5,TABLE_QA,}o,--,o{,TABLE_AUTOMATION,NaN


In [5]:
def extract_relationships_as_erd(table):
    with open(table, 'r') as source:
        d_rows = csv.DictReader(source)
        for row in d_rows:
            entry = "{} {}{}{} {} : {}".format(
                row['first-entity'],
                row['left-rel'],
                row['line'],
                row['right-rel'],
                row['second-entity'], 
                '""')
            print(entry)

In [6]:
def convert_to_erd(table):
    gs = re.search(r'\/(?P<table_name>[A-Z_]+)\.csv', table)
    root_name = gs.groupdict()['table_name']
    with open(table, 'r') as source:
        d_rows = csv.DictReader(source)
        print("{} {}".format(root_name, "{"))
        for row in d_rows:
            print("    {}  {}".format(row['Type'], row['Name']))
        print("}")

In [8]:
print("erDiagram")
extract_relationships_as_erd('tables/_RELATIONSHIPS.csv')
for table in the_tables:
    convert_to_erd(table)

erDiagram
PROJECTS }o--o{ APP : ""
APP }o--o{ GROUP : ""
GROUP }o--o{ TABLE : ""
GROUP }o--o{ TABLE_AUTOMATION : ""
TABLE_AUTOMATION }o--o{ TABLE_QA : ""
TABLE_QA }o--o{ TABLE_AUTOMATION : ""
TABLE {
    string  column
    string  description
    string  nullable
    string  unique
    string  format
    string  active
}
TABLE_QA {
    string  table
    string  column
    string  test_id
    string  description
    string  priority
}
TABLE_AUTOMATION {
    string  column
    string  format
    string  nullable
    string  length
    string  type
    string  enumeration
}
GROUP {
    string  group
    string  table_name
    string  format
}
PROJECTS {
    string  project
    string  vpn
    string  s3_output_path
    string  app
}
APP {
    string  app
    string  group
}
